<a href="https://colab.research.google.com/github/Jonathan-Oviedo/ML-23-TEA/blob/Labs/Jon_O_Bach_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bach Project

Jonathan Oviedo

# Importing the needed libraries





In [7]:
from pandas import DataFrame, Series
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn import tree

#First attempt: Bagging
results 74.05%


# Reading the given csv

In [4]:
bach = pd.read_csv('bach.csv')
bach

,choral_ID,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,000106b_,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
1,000106b_,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
2,000106b_,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
3,000106b_,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
4,000106b_,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,015505b_,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
5661,015505b_,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
5662,015505b_,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M
5663,015505b_,108,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3,C_M


#Preparing Data

droping the columns that are not needed.

In [5]:
bach.drop(columns='choral_ID', inplace=True)
bFeatures = bach.drop('chord_label', axis=1)
bLabels = bach['chord_label']

importing encoder and fitting it it to bachENC


In [6]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
bachENC = enc.fit_transform(bFeatures)
bachENC

<5665x252 sparse matrix of type '<class 'numpy.float64'>'
	with 84975 stored elements in Compressed Sparse Row format>

Divide into training and testing sets

In [9]:
#Dividing into training and test sets
bach_train_features, bach_test_features, bach_train_labels, bach_test_labels = train_test_split(bachENC, bLabels, test_size = 0.2, random_state=42)

Finding and settings the parameters for bagging classifier.

In [10]:
#treeCLF
clf = tree.DecisionTreeClassifier(criterion='entropy')

#Defining the parameter
paramDistribution = {
    'n_estimators': [50, 100, 200, 500],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
#creating an rfc
RFC = RandomForestClassifier()

#settings the rdc to the bagging classifier
baggingCLF = BaggingClassifier(clf, n_estimators=20, max_samples=100, 
                                bootstrap=True, n_jobs=-1)
#creating a randomSearch
randomSearch = RandomizedSearchCV(RFC, paramDistribution, n_iter=10, cv=5)

#fitting the randomSearch
randomSearch.fit(bach_train_features, bach_train_labels)

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'max_depth': [None, 5, 10, 20],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200, 500]})

Getting the best params

In [12]:
#getting params from the fitting
randomSearch.best_params_
#setting those params to a variable
bestParams = randomSearch.best_params_

In [13]:
#getting the prediciton
prediction = randomSearch.predict(bach_test_features)

In [14]:
#getting and displaying the accuracy
accuracy = accuracy_score(bach_test_labels, prediction)
accuracy

0.7405119152691968

# Best Accuracy Score I got with Bagging was:

 74.05%



#Second Attempt: XGBoost
Accuracy 73.41%

# Reading the given csv like before


In [21]:
bach = pd.read_csv('bach.csv')
bach

,choral_ID,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,000106b_,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
1,000106b_,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
2,000106b_,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
3,000106b_,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
4,000106b_,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,015505b_,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
5661,015505b_,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
5662,015505b_,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M
5663,015505b_,108,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3,C_M


#Preparing the data

In [22]:
#removing event# and choralID
bach.drop(columns='event_number', inplace=True)
bach.drop(columns='choral_ID', inplace=True)
#displaying bach to check if columns were dropped
bach

,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
1,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
3,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
5661,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
5662,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M
5663,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3,C_M


In [26]:
counts = bach['chord_label'].value_counts()

# select only the rows where the value appears once
value = counts[counts < 2].index.tolist()

#rows where the value appears once is added to bachInstance and displayed
bachInstance = bach[bach['chord_label'].isin(value)]

#values stored in bachInstance are then dropped if they appear only once.
bach = bach[~bach.isin(bachInstance)].dropna()

#like before for bagging we drop the chord label for features after removing the instances that appear once
bachFeatures = bach.drop('chord_label', axis=1)
bachLabels = bach['chord_label']

Importing encoder and fitting to bachENC 
also creating a label encoder and fitting to bachLabels

In [27]:
enc = OneHotEncoder(handle_unknown='ignore')
bachENC = enc.fit_transform(bFeatures)
#creating labelEncoder
le = LabelEncoder()
bachLabel = le.fit_transform(bachLabels)


Divide into training and test sets

In [31]:
#Dividing intro training and test sets
bach_train_features, bach_test_features, bach_train_labels, bach_test_labels = train_test_split(bachENC, bachLabel, test_size = 0.2, random_state=42, stratify=bachLabel)

Finding parameters for xgBoost unlike the previous parameters this attempt took a slightly longer time than usual. Take into account before running

In [34]:
from xgboost import XGBClassifier

params = { 'tree_method':'gpu_hist', 'predictor':'gpu_predictor'}
#setting model to the xgb parameteres found
model = XGBClassifier(**params)


#Defining the parameter
paramDistribution = {
    'n_estimators': [50, 100, 200, 500],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
#setting comb and folds for the stratifiedKfold
param_comb = 5
folds=5
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
#creating random search using parameters
randomSearch = RandomizedSearchCV(model, param_distributions=paramDistribution, n_iter=param_comb,  n_jobs=-1, 
                                   cv=skf.split(bach_train_features,bach_train_labels), verbose=3)
#fitting randomSearch
randomSearch.fit(bach_train_features, bach_train_labels)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


[02:42:05] WARNING: ../src/learner.cc:767: 
Parameters: { "min_samples_leaf", "min_samples_split" } are not used.



RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x7fd585f2a040>,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type...
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           predictor='gpu_predictor',
                                           random_state=None, ...),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'max_depth': [None, 5, 10, 20],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200, 500]},
                   verbose=3)

Getting the best parameters

In [35]:
#getting params from the fitting
randomSearch.best_params_
#setting those params to a variable
bestParams = randomSearch.best_params_

In [36]:
#getting the prediciton
predictions = randomSearch.best_estimator_.predict(bach_test_features)

In [37]:
#getting and displaying the accuracy
Accuracy = accuracy_score(bach_test_labels, predictions)
Accuracy

0.7340989399293286

#Score I got using XGBoost was:


  73.41%

## Final Results

 The best accuracy I was able to achieve, from either XGBoost or Bagging, was 74.05%. That accuracy came from Bagging using randomsearch. I was unable to achieve a higher score using xgboost and I believe that to improve I may want to attempt using grid search instead of randomsearch.